In [1]:
import extract as e
from tqdm import tqdm
import logging
import pandas as pd
from pygrametl.datasources import CSVSource, SQLSource
from datetime import datetime
from El_Módulo import color_print

# Configure logging
logging.basicConfig(
    filename='cleaning.log',           # Log file name
    level=logging.INFO,           # Logging level
    format='%(message)s'  # Log message format
)


def build_dateCode(date) -> str:
    return f"{date.year}-{date.month}-{date.day}"


def build_monthCode(date) -> str:
    return f"{date.year}{str(date.month).zfill(2)}"

¡Conectado!


`diccionario['AIMS.flights]` es el iterador que va devolviendo cada vuelo. Cada uno de sus elementos es un diccionario con esto: 

`{
    'id': int, 

    'aircraftregistration': str, 

    'scheduleddeparture': datetime, 

    'scheduledarrival': datetime, 

    'kind': str, 
    
    'flightid': str, 

    'departureairport': str, 

    'arrivalairport': str, 

    'actualdeparture': datetime, 

    'actualarrival': datetime, 

    'cancelled': bool, 

    'delaycode': None, 

    'passengers': 138, 

    'cabincrew': 3, 

    'flightcrew': 3
}`


In [2]:
diccionario = e.extract()

diccionario

{'AIMS.flights': <pygrametl.datasources.SQLSource at 0x2ab8315dd90>,
 'AIMS.maintenance': <pygrametl.datasources.SQLSource at 0x2ab83342290>,
 'AIMS.slots': <pygrametl.datasources.SQLSource at 0x2ab6b0568d0>,
 'AMOS.attachments': <pygrametl.datasources.SQLSource at 0x2ab67c10650>,
 'AMOS.forecastedorders': <pygrametl.datasources.SQLSource at 0x2ab81aa9f10>,
 'AMOS.maintenanceevents': <pygrametl.datasources.SQLSource at 0x2ab6b185dd0>,
 'AMOS.operationinterruption': <pygrametl.datasources.SQLSource at 0x2ab6b187550>,
 'AMOS.postflightreports': <pygrametl.datasources.SQLSource at 0x2ab6b01de10>,
 'AMOS.technicallogbookorders': <pygrametl.datasources.SQLSource at 0x2ab8334f610>,
 'AMOS.workorders': <pygrametl.datasources.SQLSource at 0x2ab8334f3d0>,
 'AMOS.workpackages': <pygrametl.datasources.SQLSource at 0x2ab8303bfd0>}

In [ ]:
def transform( diccionario: dict[str, SQLSource] ):

    # Table AIMS.flights
    source_flights = diccionario['AIMS.flights']

    #Es un diccionario donde la clave es (aircraft, day) y el contenido es un diccionario con métricas necesarias para calcular los kpis
    metrics_dict:dict[ (str, str), dict ] = {} 


    for i, flight in enumerate(source_flights):
        
        # Get aircraft and day
        aircraft:str = flight['aircraftregistration']
        day:str = build_dateCode(flight['scheduleddeparture'])
        key = (aircraft, day)
        print(f"\nVuelo {i}: Hecho por el avión {aircraft} el día {day}")
        

        #Compute flight_hours
        try: 
            flight_hours_datetime:datetime = flight['actualarrival'] - flight['actualdeparture']
            flight_hours:float = flight_hours_datetime.total_seconds() / 3600
            col = 0

            if flight_hours < 0: col = 1
        except: 
            flight_hours = 0
            col = 2
        
        color_print(f"    Actual departure = {flight['actualdeparture']}, Actual arrival = {flight['actualarrival']} ---> Flight hours = {flight_hours}", col)


        #Compute cancellations
        cancelled = flight['cancelled']
        delay_time_datetime = flight['actualdeparture'] - flight['scheduleddeparture']
        delay_time:float = delay_time_datetime.total_seconds() / 60




        # Update value on metrics_dict
        if key not in metrics_dict:
            metrics_dict[key] = {'flight_hours': 0, 'cycles': 0}
        metrics_dict[ (aircraft, day) ]['flight_hours'] += 1


        if i > 40: break


diccionario = e.extract()
transform(diccionario)


Vuelo 0: Hecho por el avión XY-ACK el día 2023-1-1
    Actual departure = 2023-01-01 03:28:56, Actual arrival = 2023-01-01 04:28:56 ---> Flight hours = 1.0

Vuelo 1: Hecho por el avión XY-ACK el día 2023-1-1
    Actual departure = 2023-01-01 06:38:40, Actual arrival = 2023-01-01 08:38:40 ---> Flight hours = 2.0

Vuelo 2: Hecho por el avión XY-ACK el día 2023-1-2
    Actual departure = 2023-01-02 04:42:43, Actual arrival = 2023-01-02 08:42:43 ---> Flight hours = 4.0

Vuelo 3: Hecho por el avión XY-ACK el día 2023-2-17
    Actual departure = 2023-02-17 20:23:21, Actual arrival = 2023-02-17 23:23:21 ---> Flight hours = 3.0

Vuelo 4: Hecho por el avión XY-ACK el día 2023-2-18
    Actual departure = 2023-02-18 11:46:17, Actual arrival = 2023-02-18 13:46:17 ---> Flight hours = 2.0

Vuelo 5: Hecho por el avión XY-ACK el día 2023-2-19
    Actual departure = 2023-02-19 06:01:46, Actual arrival = 2023-02-19 10:01:46 ---> Flight hours = 4.0

Vuelo 6: Hecho por el avión XY-ACK el día 2023-2-19
  

In [13]:
diccionario = e.extract()
source_flights = diccionario['AIMS.flights']
yeah = iter(source_flights)

#Procesar un vuelo solo y ver qué tal: 
flight = next(yeah)

flight

{'id': 1,
 'aircraftregistration': 'XY-ACK',
 'scheduleddeparture': datetime.datetime(2023, 1, 1, 3, 23, 56),
 'scheduledarrival': datetime.datetime(2023, 1, 1, 4, 23, 56),
 'kind': 'Flight',
 'flightid': '230101-TIA-GVA-4486-XY-ACK',
 'departureairport': 'TIA',
 'arrivalairport': 'GVA',
 'actualdeparture': datetime.datetime(2023, 1, 1, 3, 28, 56),
 'actualarrival': datetime.datetime(2023, 1, 1, 4, 28, 56),
 'cancelled': False,
 'delaycode': None,
 'passengers': 138,
 'cabincrew': 3,
 'flightcrew': 3}

In [32]:
flight_hours = flight['actualarrival'] - flight['actualdeparture']

fh = flight_hours.total_seconds() / 3600

type(fh)

float